In [2]:
import pandas as pd
import numpy as np


In [1]:
data_path = 'C:\\Users\\Adaku\\Documents\\Authorship-Attribution-for-Neural-Text-Generation\\data\\'

# Load Dataset

In [3]:
ctrl = pd.read_csv(data_path + 'new_ctrl.csv')
gpt = pd.read_csv(data_path + 'new_gpt.csv')
gpt2 = pd.read_csv(data_path + 'new_gpt2.csv')

grover = pd.read_csv(data_path + 'new_grover.csv')
xlm = pd.read_csv(data_path + 'new_xlm.csv')
xlnet = pd.read_csv(data_path + 'new_xlnet.csv')

pplm = pd.read_csv(data_path + 'new_pplm.csv')
human = pd.read_csv(data_path + 'new_human.csv')
em_lm = pd.read_csv(data_path + 'new_fair.csv')

In [4]:
ctrl = ctrl.drop(['Unnamed: 0'], axis=1)


In [5]:
gpt = gpt.drop(['Unnamed: 0'], axis=1)
gpt2 = gpt2.drop(['Unnamed: 0'], axis=1)

grover = grover.drop(['Unnamed: 0'], axis=1)
xlm = xlm.drop(['Unnamed: 0'], axis=1)

xlnet = xlnet.drop(['Unnamed: 0'], axis=1)
pplm = pplm.drop(['Unnamed: 0'], axis=1)

fair = em_lm.drop(['Unnamed: 0'], axis=1)


In [6]:
human = human.drop(['Unnamed: 0'], axis=1)

In [7]:
ctrl.head()

,Generation,label
0,latest headlines on cnn business tl;dr the u.s...,ctrl
1,china wants to take a victory lap over its han...,ctrl
2,coronavirus disinformation creates challenges ...,ctrl
3,china coronavirus: eating wild animals made il...,ctrl
4,china's economy could shrink for the first tim...,ctrl


In [8]:
len(human)

1066

# Build Data

In [9]:
generate = pd.concat([ctrl, gpt, gpt2, grover, xlm, xlnet, pplm, human, em_lm])

In [10]:
generate.head()

,Generation,label,Unnamed: 0
0,latest headlines on cnn business tl;dr the u.s...,ctrl,NaN
1,china wants to take a victory lap over its han...,ctrl,NaN
2,coronavirus disinformation creates challenges ...,ctrl,NaN
3,china coronavirus: eating wild animals made il...,ctrl,NaN
4,china's economy could shrink for the first tim...,ctrl,NaN


In [11]:
Class = []

for i in generate['label']:
    if i == 'human':
        Class.append(0)
    else:
        Class.append(1)

In [12]:
len(Class)

9594

In [13]:
generate.head()

,Generation,label,Unnamed: 0
0,latest headlines on cnn business tl;dr the u.s...,ctrl,NaN
1,china wants to take a victory lap over its han...,ctrl,NaN
2,coronavirus disinformation creates challenges ...,ctrl,NaN
3,china coronavirus: eating wild animals made il...,ctrl,NaN
4,china's economy could shrink for the first tim...,ctrl,NaN


# Save built dataset

In [14]:
hvm = pd.DataFrame({'text': generate['Generation'], 'class': Class})
hvm.to_csv('hvm.csv')

In [15]:
len(generate)

9594

# Build individual Human-Machine subset datasets

In [16]:
ctrl_human = pd.concat([ctrl, human])
gpt_human = pd.concat([gpt, human])

gpt2_human = pd.concat([gpt2, human])
grover_human = pd.concat([grover, human])

xlm_human = pd.concat([xlm, human])
xlnet_human = pd.concat([xlnet, human])

pplm_human = pd.concat([pplm, human])
fair_human = pd.concat([fair, human])

In [17]:
ctrl_human['label'].unique()

array(['ctrl', 'human'], dtype=object)

In [18]:
fair_human.head()

,Generation,label
0,latest headlines on cnn business : u.s. unempl...,fair
1,china wants to take a victory lap over its han...,fair
2,coronavirus disinformation creates challenges ...,fair
3,china coronavirus: eating wild animals made il...,fair
4,china's economy could shrink for the first tim...,fair


In [19]:
ctrl_human.to_csv(data_path + 'ctrl_human.csv')
gpt_human.to_csv(data_path + 'gpt_human.csv')

gpt2_human.to_csv(data_path + 'gpt2_human.csv')
grover_human.to_csv(data_path + 'grover_human.csv')

xlm_human.to_csv(data_path + 'xlm_human.csv')
xlnet_human.to_csv(data_path + 'xlnet_human.csv')

pplm_human.to_csv(data_path + 'pplm_human.csv')
fair_human.to_csv(data_path + 'fair_human.csv')


# Build balanced dataset

In [54]:
balanced = pd.concat([ctrl.sample(133), pplm.sample(133), gpt.sample(134), gpt2.sample(133), grover.sample(134), 
                     xlm.sample(133), xlnet.sample(133), em_lm.sample(133), human])

In [55]:
balanced.head()

,Unnamed: 0,Generation,label
964,964,bbc london the british government is expected ...,ctrl
207,207,workers fight for rights at ground zero for us...,ctrl
969,969,entrepreneurship and the arts the museum of mo...,ctrl
671,671,in coronavirus crisis tom hanks is more of a r...,ctrl
275,275,zunar and fahmi reza the cartoonists who helpe...,ctrl


In [56]:
balanced_class = []
for i in balanced['label']:
    if i == 'human':
        balanced_class.append(0)
    else:
        balanced_class.append(1)

In [57]:
unique_elements, counts_elements = np.unique(balanced_class, return_counts=True)

In [58]:
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1066 1066]]


In [59]:
balanced_data = pd.DataFrame({'text': balanced['Generation'], 'class': balanced_class})

In [60]:
balanced_data.to_csv('balanced_p2.csv')

# Human vs. Machine Binary Classifier

In [85]:

import numpy as np
import pandas as pd 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


In [87]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [88]:
from yellowbrick.classifier import ClassPredictionError
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

C:\Users\Adaku\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [39]:
def classify(data, label):
            
        
    X_train, X_test, y_train, y_test = train_test_split(data, label, stratify = label, test_size = 0.2, random_state = 1234)
    
    clf = RandomForestClassifier(random_state=1234,n_estimators=150,n_jobs=-1)    
    v = TfidfVectorizer()
    
    train_corpus = X_train
    test_corpus = X_test
    
    train_vector = v.fit_transform(train_corpus)
    test_vector = v.transform(test_corpus)
    
#     # Instantiate the classification model and visualizer
#     visualizer = ClassPredictionError(clf, classes= [0,1], encoder={0: 'HUMAN', 1: 'MACHINE'})

#     # Fit the training data to the visualizer
#     visualizer.fit(train_vector, y_train)

#     # Evaluate the model on the test data
#     visualizer.score(test_vector, y_test)

#     # Draw visualization
#     visualizer.show()
    
    

    fit = clf.fit(train_vector,y_train)
    pred = clf.predict(test_vector)
    
    matrix = confusion_matrix(y_test, pred, labels = [0, 1])
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print('confusion matrix: ', mat)
    
    Accuracy = accuracy_score(y_test,pred)
    F1 = f1_score(y_test, pred, average='macro')
    print("Accuracy:", Accuracy)
    print('F1:', F1)
    
    rec = recall_score(y_test, pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, pred, average='macro')
    print('Precision: ', prec)
    
    return y_test, pred

In [51]:
classify(balanced_data['text'], balanced_data['class'])

confusion matrix:  [0.97652582 0.86915888]
Accuracy: 0.9227166276346604
F1: 0.9225109296378784
Recall:  0.9228423500504586
Precision:  0.9275889608660928


(197    0
 141    1
 0      1
 482    0
 44     1
       ..
 764    1
 831    1
 980    0
 377    0
 869    1
 Name: class, Length: 427, dtype: int64,
 array([0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 

In [40]:
classify(generate['Generation'], Class)

confusion matrix:  [0.18309859 1.        ]
Accuracy: 0.9093277748827514
F1: 0.6305008897033437
Recall:  0.5915492957746479
Precision:  0.9537234042553191


([1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,


In [90]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 0.22.1.
